# Transition Tuples for Covid + Patients

In [38]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import pickle
import os

In [6]:
pth = "/tigress/BEE/penn-covidsub/wip-state_rep_all_feat_24H.csv"
pth = '/tigress/BEE/penn-covidsub/COVID - boosting_24H.csv'

In [10]:
allFrames = pd.read_csv(pth)
state_feats = ['ALBUMIN', 'ANION GAP',
       'BASE EXCESS', 'BICARBONATE', 'BILIRUBIN', 'CALCIUM',
       'CARBOXYHEMOGLOBIN', 'CHLORIDE', 'CREATININE', 'HEMATOCRIT',
       'HEMOGLOBIN', 'INSPIRED OXYGEN', 'INTERNATIONAL NORMALIZED RATIO',
       'LACTATE', 'METHEMOGLOBIN', 'PARTIAL THROMBOPLASTIN TIME', 'PCO2', 'PH',
       'PLATELETS', 'PO2', 'POTASSIUM', 'SODIUM', 'UREA NITROGEN',
       'URINE OUTPUT', 'WHITE BLOOD CELLS', 'FIO2', 'PEEP', 'OXYGEN (L)',
       'Respiratory Aids', 'Nonsteroidal Anti-inflammatory Agents (NSAIDs)',
       'Corticosteroids - Topical', 'Mineralocorticoids',
       'Glucocorticosteroids', 'Influenza Agents', 'Antiretrovirals',
       'is_deceased_next_t']

In [14]:
allFrames

,Unnamed: 0,pat_id,order_time,ALBUMIN,ANION GAP,BASE EXCESS,BICARBONATE,BILIRUBIN,CALCIUM,CARBOXYHEMOGLOBIN,...,PEEP,OXYGEN (L),Respiratory Aids,Nonsteroidal Anti-inflammatory Agents (NSAIDs),Corticosteroids - Topical,Mineralocorticoids,Glucocorticosteroids,Influenza Agents,Antiretrovirals,is_deceased_next_t
0,0,69036d47ac,2020-03-03,-1.0,-1,-21.5,-1.0,0.2,8.8,-1.0,...,-1,-1,0,0,0,0,0,0,0,0
1,1,69036d47ac,2020-03-04,-1.0,-1,-5.1,-1.0,-1.0,8.0,-1.0,...,-1,-1,0,0,0,0,0,0,0,0
2,2,69036d47ac,2020-03-05,-1.0,-1,-1.0,-1.0,-1.0,8.3,-1.0,...,-1,-1,0,0,0,0,0,0,0,0
3,3,69036d47ac,2020-03-15,-1.0,-1,-10.6,-1.0,0.4,8.6,-1.0,...,-1,-1,0,0,0,0,0,0,0,0
4,4,69036d47ac,2020-03-16,-1.0,-1,-10.5,-1.0,-1.0,8.5,-1.0,...,-1,-1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18920,18920,dfdc8dde8f,2020-04-27,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1,-1,0,1,0,0,0,0,0,0
18921,18921,dfdc8dde8f,2020-04-28,-1.0,-1,-1.0,-1.0,0.3,8.8,-1.0,...,-1,-1,0,0,0,0,0,0,0,0
18922,18922,0bd743c76b,2020-06-29,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1,-1,0,1,0,0,0,0,0,0
18923,18923,0bd743c76b,2020-06-30,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1,-1,0,0,0,0,0,0,0,0


In [49]:
def state_transformer(frames, states=None, feats=state_feats, transformer='tuples/transformer.pkl'):
   
    print('#:', len(feats))
    if states is None:
        states = np.vstack([np.array(frames.loc[i, feats]).astype(float) for i in range(len(frames))])
    if False: #os.path.isfile(transformer):
        scaler = pickle.load(open(transformer, 'rb'))
    else:
        scaler = preprocessing.StandardScaler().fit(states)
        pickle.dump(scaler, open('tuples/transformer.pkl', 'wb'))
    transformed_states = scaler.transform(states)
    return transformed_states

def state_invtransformer(tstates, transformer='tuples/transformer.pkl'):
    
    scaler = pickle.load(open(transformer, 'rb'))
    states = scaler.inverse_transform(tstates)
    
    return states

def transform(state, transformer='tuples/transformer.pkl'):
    if os.path.isfile(transformer):
        scaler = pickle.load(open(transformer, 'rb'))
        return scaler.transform([state])[0]
    else:
        return state
    return 

def discretize(a, el='Respiratory Aids'):
    
    print("A ", a )
    if el=='Respiratory Aids':
        adict = {'none': 0, 'vent': 0}
        
        # What is ivd, ivh?
        aid = a[0]
        
        if aid:
            adict['vent'] = 1        
        
    da = list(adict.values())
    if sum(da) == 0: da[0] = 1

    return da

def reward(s, a, ns, w=np.array([1, 1, 1, 1, 1])/5., el='K'):
    
    rdict = {'deceased': 0, 'cost-po': 0, 'high': 0, 'low': 0, 'other': 0}
    
    if a[0] > 0 : rdict['cost-iv'] -= 1 
    if a[1] > 0 : rdict['cost-po'] -= 1
    
    if el=='K': rdict['high'], rdict['low'] = sigmoid(ns[0], el=el)
    if el=='Mg': rdict['high'], rdict['low'] = sigmoid(ns[1], el=el)
    if el=='P': rdict['high'], rdict['low'] = sigmoid(ns[2], el=el)
    # What does this mean? And-ing a set of floats
#     if el == 'K':
#         print(str(s[30]))
#         print(str(s[31]))
#         print(str(s[32]))
#         rdict['other'] = -1 * (s[30] & s[31] & s[32])
    
    phi = np.array(list(rdict.values()))
    r = np.dot(phi, w)
    
    return phi, r

def sigmoid(x, el='K'):
    
    minmax = {'K': [3.5, 4.5], 'Mg': [1.5, 2.5], 'P': [2.5, 4.5]}
    lmin, lmax = minmax[el]
    
    if x < lmin:
        z = 1/(1 + np.exp(-3.5*(x-(lmin-1)))) - 1
        return (0, z)
    elif x > lmax: 
        z = - 1/(1 + np.exp(-3.5*(x-(lmax+1))))
        return (z, 0)
    else:
        z = 0
        return (z, z)
    
def generate_samples(vnum, trainFrames, el='Respiratory Aids'):

    frame = trainFrames[trainFrames.pat_id==vnum]
    all_st = []
    all_nst = []
    all_a = []
    all_phi = []
    all_r = []

    for i in frame.index[:-1]:
        
        s = list(frame.loc[i, state_feats])
        st = transform(s)
        all_st.append(st)
        if el=='Respiratory Aids':
            a = list(frame.loc[i+1, ['Respiratory Aids']])
        da = discretize(a, el=el)
        all_a.append(da)
        ns = list(frame.loc[i+1, state_feats])
        nst = transform(ns)
        all_nst.append(nst)
        phi, r = reward(s, a, ns, el=el) 
        all_phi.append(phi)
        all_r.append(r)
        #print('s:', s, '\n\na:', a, '\n\nns', ns, '\n\nr', phi, r)
        
    return (all_st, all_a, all_nst, all_phi, all_r)

def combine(ent):
    return np.concatenate(np.array(ent))

def get_tuples(frames, filename='tuples.pkl', el='Respiratory Aids'):
    
    transition_tuples = {'s': [], 'a': [], 'ns': [], 'phi': [], 'r': [], 'vnum': []}
    
    if el=='Respiratory Aids':
        visits = frames[(frames['Respiratory Aids']!=0)].pat_id.unique()
        
    for vnum in visits:
        if len(frames[frames.pat_id==vnum]) > 1:
            s, a, ns, phi, r = generate_samples(vnum, frames, el)
            transition_tuples['s'].append(np.array(s))
            transition_tuples['a'].append(np.array(a))
            transition_tuples['ns'].append(ns)
            transition_tuples['phi'].append(phi)
            transition_tuples['r'].append(r)
            transition_tuples['vnum'].append(np.repeat(vnum, len(r)))

    for k in transition_tuples.keys():
        transition_tuples[k] = combine(transition_tuples[k])
    
    print(str(transition_tuples))
    pickle.dump(transition_tuples, open(cf.outputdir + filename, 'wb'))
    
    return transition_tuples

In [50]:
print('Total number of processed adms =', len(allFrames.pat_id.unique()),'; number of transitions =', len(allFrames))    
allFrames = allFrames.sort_values(by=['pat_id', 'order_time'])
bp = np.sort(allFrames.pat_id.unique())[75]
trainFrames = allFrames[allFrames['pat_id'] < bp].reset_index()
testFrames = allFrames[allFrames['pat_id'] >= bp].reset_index()
trainFrames.to_csv('tuples/trainFrames.csv', index=False)
testFrames.to_csv('tuples/testFrames.csv', index=False)
print('Transformer')
ts = state_transformer(trainFrames)
visits = allFrames.pat_id.unique()

Total number of processed adms = 2063 ; number of transitions = 18925
Transformer
#: 36


In [51]:
print('Training set:')
print('Total number of processed adms =',len(trainFrames.pat_id.unique()),'; number of transitions =',len(trainFrames))
print ('Respiratory Aids Cohort')
print('Number given respiratory aids =', len(allFrames[allFrames["Respiratory Aids"] != 0].pat_id.unique()))
print('Get tuples')
get_tuples(trainFrames, filename='train_resp_tuples.pkl', el='Respiratory Aids')

Training set:
Total number of processed adms = 75 ; number of transitions = 756
Respiratory Aids Cohort
Number given respiratory aids = 8
Get tuples
A  [0]


IndexError: list index out of range